**EE4685 Assignment 2: Building a miniGPT** by Josephine King and Alec Daalman

**References:**
- "Let's build GPT: from scratch, in code, spelled out." Youtube tutorial by Andrej Karpathy. https://www.youtube.com/watch?v=kCc8FmEb1nY


In [ ]:
# Import packages
import tiktoken
import torch
import torch.nn as nn
from torch.nn import functional as F

# Install and open the TinyShakespeare dataset
!wget -O tinyshakespeare.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('tinyshakespeare.txt', 'r', encoding='utf-8') as f: raw_data = f.read()

# Set the seed
torch.manual_seed(6250513)

# Initialize some model parameters
train_pct = 0.8
block_size = 8
batch_size = 4


--2025-03-04 12:23:03--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘tinyshakespeare.txt’

tinyshakespeare.txt 100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2025-03-04 12:23:03 (7.25 MB/s) - ‘tinyshakespeare.txt’ saved [1115394/1115394]



Encode the data and convert it into a PyTorch tensor. Then split into training and validation sets.

In [22]:
# Tokenize the data using the gpt2 encoding model
enc = tiktoken.get_encoding("gpt2")
tokenized_data = enc.encode(raw_data)
# Convert into a pytorch tensor
tensor_data = torch.tensor(tokenized_data, dtype=torch.long)

# Split into training and validation sets
train_end = int(len(tensor_data)*train_pct)
train_data = tensor_data[:train_end]
validation_data = tensor_data[train_end:]

Function get_batch takes in the data, batch_size, and block_size, and returns the input and output batches.

In [ ]:
def get_batch(data, batch_size, block_size):
    # Choose batch_size random starting points
    block_starts = torch.randint(0, len(data) - block_size, (batch_size,))
    # Get the inputs and outputs for the chosen blocks, stack them into tensors
    batch_inputs = torch.stack([data[start: start + block_size] for start in block_starts])
    batch_outputs = torch.stack([data[start + 1: start + block_size + 1] for start in block_starts])
    return batch_inputs, batch_outputs

inputs, outputs = get_batch(train_data, batch_size, block_size)
print(inputs)
print(outputs)


tensor([[ 1199,   577,   387,  6724,   774,  4437,    11,  8539],
        [ 6665,   284, 15403,   866,   198,  2202,  8208, 15222],
        [21140,  4252,     0,   198,  2437,   880, 22960,   340],
        [  198, 22073,   345,    11,   616, 15876,    11,   428]])
tensor([[  577,   387,  6724,   774,  4437,    11,  8539,   276],
        [  284, 15403,   866,   198,  2202,  8208, 15222,  7021],
        [ 4252,     0,   198,  2437,   880, 22960,   340,   262],
        [22073,   345,    11,   616, 15876,    11,   428,  1310]])
